### Imports etc

In [39]:
#startup template for Jupyter Notebook
# activate tab completion so I won't shy away from descriptive vars
%config IPCompleter.use_jedi = False
%config IPCompleter.greedy = True
%matplotlib inline

In [40]:
from flipside import Flipside
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import collections
import warnings
#from datetime import date, time, datetime, timedelta
import time
from dateutil.relativedelta import relativedelta
import matplotlib.dates as mdates
from IPython.display import display, HTML, Markdown, Video, Javascript, Image
# Allow for multiple outputs from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # 'last' for last output other options: (last_expr, last_expr_or_assign, 'None')   
warnings.filterwarnings('ignore')
import numpy as np
from dotenv import load_dotenv
import json

#### query_flipside  

In [45]:
%%writefile   './py/query_flipside.py'
# You need to have a .env file with 'FLIPSIDE_API_KEY' defined.
# parameters of the param objects for create_query_run may be passed. (e.g. `maxAgeMinutes = 500`)
def query_flipside(sql, url='https://api-v2.flipsidecrypto.xyz/json-rpc', **kwargs):
    load_dotenv()
    headers = {
  'Content-Type': 'application/json',
  'x-api-key': os.environ.get('FLIPSIDE_API_KEY')
}
    def create_query_run():
        
        params = [
    {
        "resultTTLHours": 1,
        "maxAgeMinutes": 0,
        "sql": sql,
        "dataSource": "snowflake-default",
        "dataProvider": "flipside"
    }
    ]
        if kwargs != {}:
            for key, value in kwargs:
                params[0][key] = value
    
        payload = json.dumps({
                "jsonrpc": "2.0",
                "method": "createQueryRun",
                "params": params,
                "id": 42,
                "dataSource": "snowflake-default",
                "dataProvider": "flipside"
                })
        
        #print('inside inner function')
        try:
            response = requests.request("POST", url, headers=headers, data=payload)
            if response.status_code != 200 :
                print (f"API call received status: {response.status_code}. Exiting")
                return False
                # print(f"Status code: {response.status_code}")
            # print(f"Response text: {response.text}")
        except: 
            print("error with API request in create_query_run")
            return False
        #print(response.text)
        try: 
            if response.json()['result']['queryRun']['state'] == "QUERY_STATE_READY":
                query_run_id = response.json()['result']['queryRun']['id']
                print(json.dumps(response.json()['result']['queryRequest'], indent=4))
                return query_run_id
            else:
                print("Query state is not ready")
                return False
        except MemoryError: 
            #print(f"error with api request: {response.json()['error']['message']}")
            #print("response object has no result key")
            #print(response.text)
            return False

    
    ################################################################################
    #
    # Get Query Run 
    #
    ################################################################################
    
    def get_query_run():
        load_dotenv()
        payload = json.dumps({
          "jsonrpc": "2.0",
          "method": "getQueryRun",
          "params": [
            {
              "queryRunId": query_run_id
            }
          ],
          "id": 1
        })
        done = False
        count = 0
        time_start = time.time()
        # %%writefile -a  './py/query_flipside.py'
        while not done:
            response = requests.request("POST", url, headers=headers, data=payload)
            state = response.json()['result']['queryRun']['state']
            if state == "QUERY_STATE_SUCCESS":
                print("Success.")
                return True 
            if state != "QUERY_STATE_RUNNING":
                print(f"State {state} occured")
                return False
            if count % 10 = 0:
                print(f"status is 'running' count = {count}")
            count += 1
            time.sleep(3)
            run_time = time.time() - time_start
            if run_time  > 1000:
                print("Taking to long. Exiting.")
                return False
                
            
            
            #print(json.dumps(response.json(), indent=4))
     ################################################################################
    #
    # Get Query Run Results
    #
    ################################################################################
    
    def get_query_run_results():
        payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "getQueryRunResults",
  "params": [
    {
      "queryRunId": query_run_id,
      "format": "json",
      "page": {
        "number": 1,
        "size": 100
      }
    }
  ],
  "id": 1
    })
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.json()
        
    ################################################################################
    #
    # Main Body of query_flipside
    #
    ################################################################################
    
    query_run_id = create_query_run()
    if not query_run_id:
        return False
    else:
        print(f"Ready to send query_id '{query_run_id}' to `get_query_run`")
        if get_query_run():
            print("Query success. Ready to get results!")
            results = get_query_run_results()
            df = pd.DataFrame(results['result']['rows'])
            return df
        
 
    

Writing ./py/query_flipside.py


In [42]:
## test script for making sure api is working
# sql = """
# SELECT 
#   date_trunc('hour', block_timestamp) as hour,
#   count(distinct tx_hash) as tx_count
# FROM ethereum.core.fact_transactions 
# WHERE block_timestamp >= GETDATE() - interval'7 days'
# GROUP BY 1
# """
#query_flipside(sql)

{
    "id": "clxnc9kd90g1zna0tjyjkmww8",
    "sqlStatementId": "clxmkdlmq3ja6ou0t4pyicz9w",
    "userId": "clgztx7jw00qxob0sc634x5rg",
    "tags": {},
    "maxAgeMinutes": 0,
    "resultTTLHours": 1,
    "userSkipCache": true,
    "triggeredQueryRun": true,
    "queryRunId": "clxnc9kcm0g1xna0th8kno7r2",
    "createdAt": "2024-06-20T14:09:47.000Z",
    "updatedAt": "2024-06-20T14:09:47.000Z"
}
Ready to send query_id 'clxnc9kcm0g1xna0th8kno7r2' to `get_query_run`
status is 'running' count = 0
status is 'running' count = 1
status is 'running' count = 2
status is 'running' count = 3
status is 'running' count = 4
status is 'running' count = 5
status is 'running' count = 6
status is 'running' count = 7
status is 'running' count = 8
status is 'running' count = 9
status is 'running' count = 10
Taking to long. Exiting.
status is 'running' count = 11
Taking to long. Exiting.
status is 'running' count = 12
Taking to long. Exiting.
status is 'running' count = 13
Taking to long. Exiting.
status is 

,dst_chain_id,dst_chain,__row_index
0,56,bnb,0
1,42161,arbitrum,1
2,100,gnosis,2
3,66,Unknown,3
4,250,fantom,4
5,25,Unknown,5
6,122,Unknown,6
7,5000,mantle,7
8,288,Unknown,8
9,10,optimism,9


### Chain Ids table

The below SQL script maps the chain ids used by Li.Fi to the human readable chain label. The choice of the polygon event logs table was arbitrary.

In [43]:
chainIDs_df = pd.read_csv('data/chainIDs.csv')
chainIDs_df

,DST_CHAIN,CHAIN
0,42161,arbitrum
1,56,bnb
2,100,gnosis
3,250,fantom
4,324,zksync
5,10,optimism
6,59144,linea
7,5000,mantle
8,1088,Unknown
9,81457,Unknown


We won't be using that because we are just going to incorporate it into our SQL query.

### Building the raw table. 

Using a simple loop through the list of blockchains that we are interested in and `fstrings`, we can generate the script we need by unioning all the otherwise identical queries. Each blockchain has its own schema, with the same table structures. We will be able to easily reconstruct the query, by just editing our model instead of every CTE in the script.

### Components

 - Chain ID helper table (as seen above) First CTE of our query
 - For each chain we query three tables
    - the `<chain name>.core.ez_decoded_event_logs` table 
    - the `<chain name>core.ez_native_transfers` table for native token transfers
    - the `<chain name>core.ez_transfers` table for erc20 transfers
 - We first query the `log_events` table and join it with the `chainIDs` table, for each blocckchain, assigning the value '<blockchain>' to a `chain` column (e.g. select 'bsc' as chain)
 - We then union all the tables.
 - Next we iterate through the chains again, this time joining together the results from the native and erc20 transfer tables. We label the source_token as 'native' when querying the `native_token_transfers` table.
 - Finally, we join together the bridge data with the transfer data.


### The SQL

The first part of the querry string, `sql` is the query for the `chainIDs` table

In [57]:
def get_sql_string(filename):
    with open(filename, 'r') as f:
        sql = f.read()
    return sql 

In [58]:
sql = get_sql_string('Scripts/sql/chainIDs.sql')
print(sql)

with chain_ids as (
select distinct decoded_log:bridgeData:destinationChainId as dst_chain_ID,
                case
                  when dst_chain_ID = 1 then 'ethereum'
                  when dst_chain_ID = 8453 then 'base'
                  when dst_chain_ID = 137 then 'polygon'
                  when dst_chain_ID = 42161 then 'arbitrum'
                  when dst_chain_ID = 10 then 'optimism'
                  when dst_chain_ID = 56 then 'bnb'
                  when dst_chain_ID = 43114 then 'avalanche'
                  when dst_chain_ID = 1101 then 'pol zkevm'
                  when dst_chain_ID = 59144 then 'linea'
                  when dst_chain_ID = 100 then 'gnosis'
                  when dst_chain_ID = 250 then 'fantom'
                  when dst_chain_ID = 5000 then 'mantle'
                  when dst_chain_ID = 1313161554 then 'aurora'
                  when dst_chain_ID = 7777777 then 'zora'
                  when dst_chain_ID = 324 then 'zksync'
                  when 

In [47]:
query_flipside(sql)

{
    "id": "clxndvzrp0vkyoo0twhaefif6",
    "sqlStatementId": "clxmkdlmq3ja6ou0t4pyicz9w",
    "userId": "clgztx7jw00qxob0sc634x5rg",
    "tags": {},
    "maxAgeMinutes": 0,
    "resultTTLHours": 1,
    "userSkipCache": true,
    "triggeredQueryRun": true,
    "queryRunId": "clxndvzqw0vkwoo0t5vbl2tie",
    "createdAt": "2024-06-20T14:55:13.000Z",
    "updatedAt": "2024-06-20T14:55:13.000Z"
}
Ready to send query_id 'clxndvzqw0vkwoo0t5vbl2tie' to `get_query_run`
status is 'running' count = 0
status is 'running' count = 1
status is 'running' count = 2
status is 'running' count = 3
status is 'running' count = 4
status is 'running' count = 5
status is 'running' count = 6
status is 'running' count = 7
status is 'running' count = 8
status is 'running' count = 9
status is 'running' count = 10
Taking to long. Exiting.
status is 'running' count = 11
Taking to long. Exiting.
status is 'running' count = 12
Taking to long. Exiting.
status is 'running' count = 13
Taking to long. Exiting.
status is 

### query_run_results

In [48]:
def query_run_results(queryRunId):
    import requests
    import json

url = "https://api-v2.flipsidecrypto.xyz/json-rpc"

payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "getQueryRunResults",
  "params": [
    {
      "queryRunId": queryRunId,
      "format": "csv",
      "page": {
        "number": 1000,
        "size": 10000
      }
    }
  ],
  "id": 1
})
headers = {
  'Content-Type': 'application/json',
  'x-api-key': '025c9fa7-616d-40d2-8756-701e6cf43f82'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"jsonrpc":"2.0","id":1,"result":{"columnNames":["dst_chain_id","dst_chain","__row_index"],"columnTypes":["number","string","number"],"rows":[[250,"fantom",0]],"page":{"currentPageNumber":1,"currentPageSize":1,"totalRows":28,"totalPages":28},"sql":"select * from read_parquet('/data/2024/06/20/14/clxndvzqw0vkwoo0t5vbl2tie/*') offset 0 limit 1","format":"csv","originalQueryRun":{"id":"clxndvzqw0vkwoo0t5vbl2tie","sqlStatementId":"clxmkdlmq3ja6ou0t4pyicz9w","state":"QUERY_STATE_SUCCESS","path":"2024/06/20/14/clxndvzqw0vkwoo0t5vbl2tie","fileCount":1,"lastFileNumber":null,"fileNames":"clxndvzqw0vkwoo0t5vbl2tie_results.parquet","errorName":null,"errorMessage":null,"errorData":null,"dataSourceQueryId":"01b5233f-0504-fde1-3d4f-830227680bcb","dataSourceSessionId":"17257400591461618","startedAt":"2024-06-20T14:55:13.000Z","queryRunningEndedAt":"2024-06-20T14:59:53.000Z","queryStreamingEndedAt":"2024-06-20T14:59:53.000Z","endedAt":"2024-06-20T14:59:53.000Z","rowCount":28,"totalSize":"1151","tags":

In [ ]:
# sql = """
# -- forked from all_chains_bridge_data @ https://flipsidecrypto.xyz/edit/queries/41f7b717-670a-4d7b-add9-0b5d2e214b18

# -- forked from octavionotpunk / Jumper Source Chain
# --  @ https://flipsidecrypto.xyz/octavionotpunk/q/JIUX5KquZLfu/jumper-source-chainselect 

# with chain_ids as (
#           select distinct decoded_log:bridgeData:destinationChainId as dst_chain_ID,
#                 case
#                   when dst_chain_ID = 1 then 'ethereum'
#                   when dst_chain_ID = 8453 then 'base'
#                   when dst_chain_ID = 137 then 'polygon'
#                   when dst_chain_ID = 42161 then 'arbitrum'
#                   when dst_chain_ID = 10 then 'optimism'
#                   when dst_chain_ID = 56 then 'bnb'
#                   when dst_chain_ID = 43114 then 'avalanche'
#                   # when dst_chain_ID = 1101 then 'pol zkevm'
#                   when dst_chain_ID = 59144 then 'linea'
#                   when dst_chain_ID = 100 then 'gnosis'
#                   when dst_chain_ID = 250 then 'fantom'
#                   when dst_chain_ID = 5000 then 'mantle'
#                   when dst_chain_ID = 1313161554 then 'aurora'
#                   when dst_chain_ID = 7777777 then 'zora'
#                   when dst_chain_ID = 324 then 'zksync'
#                   when dst_chain_ID = 1151111081099710 then 'solana'
#                   when dst_chain_ID = 534352 then 'scroll'
#                   when dst_chain_ID = 34443 then 'mode'
#                   else 'Unknown'
#                 end as dst_chain
# from polygon.core.ez_decoded_event_logs 
# where  event_name = 'LiFiTransferStarted'
#   and block_timestamp > current_timestamp - interval '1 day'
# ),

In [ ]:
# sql = """
# -- forked from all_chains_bridge_data @ https://flipsidecrypto.xyz/edit/queries/41f7b717-670a-4d7b-add9-0b5d2e214b18

# -- forked from octavionotpunk / Jumper Source Chain
# --  @ https://flipsidecrypto.xyz/octavionotpunk/q/JIUX5KquZLfu/jumper-source-chainselect 

# with chain_ids as (
#           select distinct decoded_log:bridgeData:destinationChainId as dst_chain_ID,
#                 case
#                   when dst_chain_ID = 1 then 'ethereum'
#                   when dst_chain_ID = 8453 then 'base'
#                   when dst_chain_ID = 137 then 'polygon'
#                   when dst_chain_ID = 42161 then 'arbitrum'
#                   when dst_chain_ID = 10 then 'optimism'
#                   when dst_chain_ID = 56 then 'bnb'
#                   when dst_chain_ID = 43114 then 'avalanche'
#                   # when dst_chain_ID = 1101 then 'pol zkevm'
#                   when dst_chain_ID = 59144 then 'linea'
#                   when dst_chain_ID = 100 then 'gnosis'
#                   when dst_chain_ID = 250 then 'fantom'
#                   when dst_chain_ID = 5000 then 'mantle'
#                   when dst_chain_ID = 1313161554 then 'aurora'
#                   when dst_chain_ID = 7777777 then 'zora'
#                   when dst_chain_ID = 324 then 'zksync'
#                   when dst_chain_ID = 1151111081099710 then 'solana'
#                   when dst_chain_ID = 534352 then 'scroll'
#                   when dst_chain_ID = 34443 then 'mode'
#                   else 'Unknown'
#                 end as dst_chain
# from polygon.core.ez_decoded_event_logs 
# where  event_name = 'LiFiTransferStarted'
#   and block_timestamp > current_timestamp - interval '1 day'
# ),

  
# polygon_raw_bridge_data as (
#               select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                      decoded_log:bridgeData:bridge as bridge,
#                      decoded_log:bridgeData:integrator as integrator,
#                      event_name,
#                      tx_hash
#               from polygon.core.ez_decoded_event_logs
#               where event_name =  'LiFiTransferStarted'
#                 and block_timestamp > current_timestamp - interval '30 days'
# ),
# polygon_bridge_data as (
#   select 'polygon' as source_chain,
#          c.dst_chain,
#           r.bridge,
#          r.integrator,
#            r.event_name,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         polygon_raw_bridge_data r 
#         using
#         (dst_chain_id)
# ),
# bsc_raw_bridge_data as (
#           select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                  decoded_log:bridgeData:bridge as bridge,
#                  decoded_log:bridgeData:integrator as integrator,
#                  event_name,
#                  tx_hash
#           from bsc.core.ez_decoded_event_logs
#           where event_name =  'LiFiTransferStarted'
#             and block_timestamp > (current_timestamp - interval '30 days')
# ),
# bsc_bridge_data as (
#   select 'bsc' as source_chain,
#          c.dst_chain,
#          r.bridge,
#          r.integrator,
#          r.event_name,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         bsc_raw_bridge_data r 
#         using
#         (dst_chain_id)
# ),
# arbitrum_raw_bridge_data as (
#           select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                  decoded_log:bridgeData:bridge as bridge,
#                  decoded_log:bridgeData:integrator as integrator,
#                  event_name,
#                  tx_hash
#           from arbitrum.core.ez_decoded_event_logs
#           where event_name =  'LiFiTransferStarted'
#             and block_timestamp > current_timestamp - interval '30 days'
# ),
# arbitrum_bridge_data as (
#   select 'arbitrum' as source_chain,
#          c.dst_chain,
#          r.bridge,
#          r.integrator,
#          r.event_name,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         arbitrum_raw_bridge_data r 
#         using
#         (dst_chain_id)
# ),
# optimism_raw_bridge_data as (
#           select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                  decoded_log:bridgeData:bridge as bridge,
#                  decoded_log:bridgeData:integrator as integrator,
#                  event_name,
#                  tx_hash
#           from optimism.core.ez_decoded_event_logs
#           where event_name =  'LiFiTransferStarted'
#             and block_timestamp > current_timestamp - interval '30 days'
# ),
# optimism_bridge_data as (
#   select 'optimism' as source_chain,
#          c.dst_chain,
#          r.bridge,
#          r.integrator,
#          r.event_name,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         optimism_raw_bridge_data r 
#         using
#         (dst_chain_id)
# ),
# base_raw_bridge_data as (
#           select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                  decoded_log:bridgeData:bridge as bridge,
#                  decoded_log:bridgeData:integrator as integrator,
#                  event_name,
#                  tx_hash
#           from base.core.ez_decoded_event_logs
#           where event_name =  'LiFiTransferStarted'
#             and block_timestamp > current_timestamp - interval '30 days'
# ),
# base_bridge_data as (
#   select 'base' as source_chain,
#          c.dst_chain,
#          r.bridge,
#          r.integrator,
#          r.event_name,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         base_raw_bridge_data r 
#         using
#         (dst_chain_id)
# ),
# avalanche_raw_bridge_data as (
#           select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                  decoded_log:bridgeData:bridge as bridge,
#                  decoded_log:bridgeData:integrator as integrator,
#                  event_name,
#                  tx_hash
#           from avalanche.core.ez_decoded_event_logs
#           where event_name =  'LiFiTransferStarted'
#             and block_timestamp > current_timestamp - interval '30 days'
# ),
# avalanche_bridge_data as (
#   select 'avalanche' as source_chain,
#          c.dst_chain,
#          r.bridge,
#          r.integrator,
#          r.event_name,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         avalanche_raw_bridge_data r 
#         using
#         (dst_chain_id)
# ),
# ethereum_raw_bridge_data as (
#           select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                  decoded_log:bridgeData:bridge as bridge,
#                  decoded_log:bridgeData:integrator as integrator,
#                  event_name,
#                  tx_hash
#           from ethereum.core.ez_decoded_event_logs
#           where event_name =  'LiFiTransferStarted'
#             and block_timestamp > current_timestamp - interval '30 days'
# ),
# ethereum_bridge_data as (
#   select 'ethereum' as source_chain,
#          c.dst_chain,
#          r.bridge,
#          r.integrator,
#          r.event_name,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         ethereum_raw_bridge_data r 
#         using
#         (dst_chain_id)
# ),


# all_chains_bridge_data as (
# select * from polygon_bridge_data
#  union all 
# select * from bsc_bridge_data
#  union all
# select * from arbitrum_bridge_data
#  union all
# select * from optimism_bridge_data
#  union all
# select * from base_bridge_data
#  union all
# select * from avalanche_bridge_data
#  union all
# select * from ethereum_bridge_data
# ),
# -- forked from all_chians_transfers_data @ https://flipsidecrypto.xyz/edit/queries/4d7fcf48-7560-447d-84b4-89eed256fdfe

#     arbitrum_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from arbitrum.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  arbitrum.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
    

#     avalanche_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from avalanche.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  avalanche.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
    

#     base_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from base.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  base.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
    

#     bsc_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from bsc.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  bsc.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
    

#     ethereum_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from ethereum.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  ethereum.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
    

#     optimism_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from optimism.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  optimism.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
    

#     polygon_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from polygon.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  polygon.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
#     all_chains_transfer_data as (
#       select * from polygon_transfers
#        union all 
#       select * from bsc_transfers
#        union all
#       select * from arbitrum_transfers
#        union all
#       select * from optimism_transfers
#        union all
#       select * from base_transfers
#        union all
#       select * from avalanche_transfers
#        union all
#       select * from ethereum_transfers
#    ),
# --select * from  all_chains_bridge_data  limit 10
# --select * from  all_chains_transfer_data  limit 10
# final as  (
# select t.block_timestamp, 
#        t.from_address,
#        round(t.amount_usd,2) as amount_usd, 
#        t.source_token,
#        b.dst_chain,
#        b.bridge,
#        b.integrator,
#       -- b.event_name
       
# from all_chains_transfer_data  t
#       join 
#      all_chains_bridge_data b
#      using(tx_hash)
#  )
# select * from final limit 5
#  """

In [ ]:
sql = """
SELECT 
  date_trunc('hour', block_timestamp) as hour,
  count(distinct tx_hash) as tx_count
FROM ethereum.core.fact_transactions 
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
"""
response_create = create_query(sql)
id = response_create['result']['queryRun']['id']
id

In [ ]:
id = 'clxcuoy0944teod0tk8tdxjt4'
query_run = run_query(id)
query_run

In [ ]:
# chains = ['arbitrum', 'bsc', 'optimism', 'base', 'avalanche', 'ethereum','polygon']
# chains.sort()
# sql = ""
# for chain in chains:
#     query[chain] = 

# sql = f"""
# {chain}_raw_bridge_data as (
#           select decoded_log:bridgeData:destinationChainId as dst_chain_id,
#                  decoded_log:bridgeData:bridge as bridge,
#                  decoded_log:bridgeData:integrator as integrator,
#                  tx_hash
#           from {chain}.core.ez_decoded_event_logs
#           where event_name =  'LiFiTransferStarted'
#             and block_timestamp > current_timestamp - interval '30 days'
# ),
# {chain}_bridge_data as (
#   select '{chain}' as source_chain,
#          c.dst_chain,
#          r.bridge,
#          r.integrator,
#          r.tx_hash
#   from chain_ids c
#         left join 
#         {chain}_raw_bridge_data r 
#         using
#         (dst_chain_id)

# """



I created the below script to facilitate a rather repetitive process, querying the tables from each chain's schema

### Transfer tables

In [ ]:
# # chains = ['arbitrum', 'bsc', 'optimism', 'base', 'avalanche', 'ethereum','polygon']
# sql = ""
# for chain in chains:
#     query = f"""
#     {chain}_transfers as (
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  'native' as source_token, 
#     	  tx_hash 
#      from {chain}.core.ez_native_transfers
#      where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#        and block_timestamp > current_timestamp - interval '30 days'
#        and amount_usd >= 20
    
#     union all
    
#     select block_timestamp, 
#     	  from_address ,
#     	  amount_usd, 
#     	  symbol as source_token, 
#     	  tx_hash 
#     from  {chain}.core.ez_token_transfers
#     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
#       and block_timestamp > current_timestamp - interval '30 days'
#       and amount_usd >= 20
#     ),
#     """
#     sql = sql + '\n' + query
# print(sql)

Almost there! We now union all these tables together

### Data collection

Columns needed: 
  - timestamp
  - receiver
  - source_chain
  - destination_chain   
  - amount usd (need to merge with price table
  - token_address
  - token symbol

In [ ]:
id